In [4]:
# Step 1: 환경 변수 및 패키지 로드
import os
import pickle
import json
from pymilvus import connections, CollectionSchema, FieldSchema, DataType, Collection
from dotenv import load_dotenv
import openai
from openai.types import CreateEmbeddingResponse, Embedding


def load_environment_variables():
    load_dotenv()
    openai.api_key = os.getenv("OPEN_API_KEY")

load_environment_variables()


In [8]:
# Step 2: Milvus 연결 및 데이터 로드
def connect_to_milvus():
    connections.connect(alias="default", host="localhost", port="19530")

def load_data(file_path):
    with open(file_path, 'rb') as file:
        return pickle.load(file)

connect_to_milvus()
raw_data = load_data('../final_result.pkl')


In [14]:
# Step 3: 데이터 처리 및 저장
def process_data(raw_data):
    return [{"question": k, "answer": v} for k, v in raw_data.items()]

def save_to_json(data, file_path):
    with open(file_path, 'w') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)

faqs = process_data(raw_data)
save_to_json(faqs, 'faqs.json')

print(f"Sample FAQ: {faqs[0]}")


Sample FAQ: {'question': '[가입절차] 스마트스토어센터 회원가입은 어떻게 하나요? (ID만들기)', 'answer': '네이버 커머스 ID 하나로 스마트스토어센터와 같은 네이버의 다양한 커머스 서비스를 편리하게 이용하실 수 있습니다.네이버 커머스 ID가 없으시다면 [가입하기] 버튼을 통해 회원가입해 주세요.\xa0\xa0 \xa0 \xa0 \xa0 \xa0\xa01. \'네이버ID\' 로 네이버 커머스 ID 가입하기1) [네이버 아이디로 가입하기] 버튼을 눌러서 네이버 로그인 해 주세요.2) 연락가능한 휴대폰번호를 입력하시고 인증하신 후 개인정보 수집동의를 체크 하여 [가입] 버튼을 눌러주세요.\xa03) 네이버 커머스 ID 회원가입이 완료되어 해당 네이버ID 로 스마트스토어센터 서비스 가입을 하실 수 있습니다.\xa0이 후 스마트스토어센터 로그인 시에 [네이버 아이디로 로그인] 버튼을 눌러서 네이버 로그인을 하실 수 있습니다.\xa02. \'이메일 아이디\'로 네이버 커머스 ID 가입하기\xa01) [이메일 아이디로 가입하기] 버튼을 눌러서 사용할 ID를 입력해 주세요.! 중요. 로그인 ID는 "실 사용중인 이메일주소"로 기재하셔야 합니다. (예 : abc@naver.com)2) 연락가능한 휴대폰번호와 이메일주소를 입력하시고 인증하신 후 개인정보 수집동의를 체크 하여 [가입] 버튼을 눌러주세요.3) 네이버 커머스 ID 회원가입이 완료되어 해당 이메일 아이디로 스마트스토어센터 서비스 가입을 하실 수 있습니다.이 후 스마트스토어센터 로그인 시에 해당 이메일 아이디와 비밀번호를 입력하여 로그인을 하실 수 있습니다.\xa0※ 참고. 로그인 ID 계정 대표자와 사업자번호 대표자 동일여부는 체크하지 않습니다. (대표자가 상이해도 가입 진행 가능합니다.)\xa0 \xa0\xa0 \xa0 \xa0 \xa0 \xa0\xa0\n\n\n위 도움말이 도움이 되었나요?\n\n\n별점1점\n\n별점2점\n\n별점3점\n\n별점4점\n\n별점5점\n\n\n\n소중한

In [9]:
# Step 4: Milvus 컬렉션 정의 및 생성
def define_collection_schema():
    fields = [
        FieldSchema(name="faq_id", dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=1536),
        FieldSchema(name="faq", dtype=DataType.JSON),
    ]
    return CollectionSchema(fields, description="FAQ collection")

def create_collection(schema):
    collection = Collection("faq_collection", schema)
    index_params = {
        "index_type": "IVF_FLAT",
        "metric_type": "IP",
        "params": {"nlist": 128}
    }
    collection.create_index(field_name="embedding", index_params=index_params)
    return collection

schema = define_collection_schema()
collection = create_collection(schema)


In [15]:
# Step 5: 임베딩 생성 및 데이터 삽입 (수정)
def generate_embeddings(questions):
    embeddings = []
    for question in questions:
        response: CreateEmbeddingResponse = openai.embeddings.create(
            model="text-embedding-ada-002",
            input=question
        )
        embedding = response.data
        embeddings.append(embedding)
    return embeddings

# 질문 및 FAQ 데이터 제한
questions = [faq["question"] for faq in faqs]
faqs_limited = faqs

# 임베딩 생성
embeddings = generate_embeddings(questions)




In [17]:

entities = [
    {
        "embedding": embedding[0].embedding,
        "faq": faq
    } for embedding, faq in zip(embeddings, faqs_limited)
]

# 데이터 삽입
collection.insert(entities)

(insert count: 2717, delete count: 0, upsert count: 0, timestamp: 454437115934015490, success count: 2717, err count: 0

In [25]:
# Step 6: 컬렉션 정보 출력
def print_collection_info(collection):
    print("컬렉션 필드 정보:")
    for field in collection.schema.fields:
        print(f"필드 이름: {field.name}, 데이터 타입: {field.dtype}")

print_collection_info(collection)


컬렉션 필드 정보:
필드 이름: faq_id, 데이터 타입: 5
필드 이름: embedding, 데이터 타입: 101
필드 이름: faq, 데이터 타입: 23


In [26]:
# Step 7: 컬렉션 쿼리 및 결과 출력
def query_collection(collection):
    collection.load()
    search_params = {"metric_type": "IP", "params": {"nprobe": 10}}
    query_vector = collection.query(
        expr="faq_id >= 0",
        output_fields=["embedding", "faq"],
        limit=5
    )
    return query_vector

def print_query_results(query_vector):
    for result in query_vector:
        print(f"임베딩 데이터: {result['embedding']}")
        print(f"FAQ 데이터: {result['faq']}")

query_vector = query_collection(collection)
print_query_results(query_vector)


임베딩 데이터: [np.float32(0.012372679), np.float32(-0.00065862626), np.float32(0.025401972), np.float32(-0.04781699), np.float32(-0.040117864), np.float32(0.016711485), np.float32(-0.021114666), np.float32(-0.0032991664), np.float32(0.0018298303), np.float32(0.0011128652), np.float32(0.014934763), np.float32(-0.0012206916), np.float32(-0.01610637), np.float32(-0.019505316), np.float32(0.0040233736), np.float32(0.020638298), np.float32(0.0076605035), np.float32(-0.005967468), np.float32(0.025543595), np.float32(-0.011941373), np.float32(-0.0075253183), np.float32(0.0065532713), np.float32(-0.004834486), np.float32(-0.012746048), np.float32(0.009405038), np.float32(-0.004638145), np.float32(0.01422665), np.float32(-0.017419599), np.float32(0.0025685218), np.float32(0.00096078165), np.float32(0.012314742), np.float32(-0.0041006226), np.float32(-0.011658128), np.float32(-0.015372506), np.float32(-0.011304071), np.float32(0.024603736), np.float32(-0.013724533), np.float32(-0.009900718), np.float

In [20]:


# Step 7: 컬렉션 쿼리 및 결과 출력
q1 = "네이버 쇼핑윈도 노출 절차는 어떻게 되나요?"
data = generate_embeddings([q1])[0][0].embedding


In [21]:
print(len(data))
search_vector = [data]
for float_data in search_vector:
    print(type(float_data))
print(type(search_vector))  # 데이터가 리스트인지 확인
print(len(search_vector))   # 리스트 길이 확인
search_param = {
    "metric_type": "IP",  # 유사도 측정 방식
    "params": {"nprobe": 10}  # 검색 정확도 조정
}

results = collection.search(
    data=search_vector,
    anns_field="embedding",  
    param=search_param,
    limit=5, 
    output_fields=["faq"]  # 반환할 추가 필드
)

# 결과 출력
for i, result in enumerate(results[0]):
    print(f"Rank {i + 1}: id={result.id}, distance={result.distance}, faq={result.entity.get('faq')}")

1536
<class 'list'>
<class 'list'>
1
Rank 1: id=454422749880753141, distance=0.9636106491088867, faq={'question': '[가입절차][쇼핑윈도/패션타운] 네이버 쇼핑윈도 노출 절차는 어떻게 되나요?', 'answer': '네이버 쇼핑 윈도에 입점하기 위해서는, [노출관리 > 쇼핑윈도 노출제안] 을 통해 자세한 조건을 확인할 수 있습니다.아래 패션타운 및 윈도 서비스는 입점 신청 전, self 입점 체크를 통해 입점 가능여부를 확인하실 수 있습니다.패션타운 해외직구 self 입점 체크 바로 가기》패션타운 뷰티 self 입점 체크 바로 가기》패션타운 소호&스트릿윈도 self 입점 체크 바로 가기》\xa0\xa0키즈윈도(로드샵)\xa0self 입점 체크 바로 가기》키즈윈도(브랜드)\xa0self 입점 체크 바로 가기》네이버펫\xa0self 입점 체크 바로 가기》리빙윈도(홈&데코/창작공방/문방구) self 입점 체크 바로 가기》\xa0입점 기준 확인 및 self 체크가 완료되었다면 아래 메뉴를 통해 입점 신청해 주세요.\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 스마트스토어센터에서\xa0입점 신청하기1) 사업자로 가입한 판매자만 입점이 가능합니다. 2) [노출관리 > 쇼핑윈도 노출제안] 메뉴를 선택하세요.3) "소호&스트릿/ 뷰티(브랜드 본사, 공식 수입사만 가능)/ 해외직구(해외직구 서비스 지역의 현지 해외 사업자)/\xa0\xa0 \xa0리빙 윈도(리빙 상품군) / 키즈 윈도(유아동) / 플레이 윈도(디지털, 스포츠/레저, 키덜트 상품군) / 아트 윈도(담당자와 사전 협의된 작가, 갤러리) /\xa0 \xa0네이버 펫 윈도(강아지/고양이 관련 브랜드 본사, 공식 수입사) / 푸드 윈도(산지직송, 지역명물, 간편집밥, 전통주 등 다양한 푸드 상품군)" 중에서 원하는 윈도를 선택해 주세요.4) 각 윈도 특성에 따라 가입 조건이 있습니다. 하단의 가입조건에 

In [16]:
## show token usage
import requests
import datetime

# OpenAI API Key
api_key = os.getenv("OPEN_API_KEY")

# 현재 날짜 및 시간
end_date = datetime.datetime.now().strftime('%Y-%m-%d')
start_date = (datetime.datetime.now() - datetime.timedelta(days=3)).strftime('%Y-%m-%d')  #

# Usage API 엔드포인트
url = f"https://api.openai.com/v1/usage?start_date={start_date}&end_date={end_date}"

# API 요청
response = requests.get(
    url,
    headers={
        "Authorization": f"Bearer {api_key}"
    }
)

# 결과 확인
if response.status_code == 200:
    usage_data = response.json()
    print("사용량 데이터:", usage_data)
else:
    print(f"API 요청 실패: {response.status_code}, {response.text}")


API 요청 실패: 400, {
  "error": {
    "message": "Missing query parameter 'date'",
    "type": "invalid_request_error",
    "param": null,
    "code": null
  }
}

